In [ ]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

BASE_URL = "https://papers.nips.cc"
HEADERS = {"User-Agent": "Mozilla/5.0"}
YEAR = 2024
MAX_PAPERS = 370

def fetch_paper_links(year):
    try:
        response = requests.get(f"{BASE_URL}/paper_files/paper/{year}", headers=HEADERS, timeout=10)
        if response.status_code != 200:
            return []
        soup = BeautifulSoup(response.text, "html.parser")
        return [a['href'] for a in soup.find_all('a', href=True) if "/paper/" in a['href']][:MAX_PAPERS]
    except requests.RequestException:
        return []

def extract_paper_details(paper_link, serial_no):
    try:
        response = requests.get(f"{BASE_URL}{paper_link}", headers=HEADERS, timeout=10)
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.text, "html.parser")
        title = (soup.find('h4').text.strip() if soup.find('h4') else "No Title")
        authors = (soup.find('h4', string="Authors").find_next('p').find('i').text.strip()
                   if soup.find('h4', string="Authors") else "No Authors")
        abstract = (soup.find('h4', string="Abstract").find_next('p').text.strip()
                    if soup.find('h4', string="Abstract") else "No Abstract")
        pdf_link = (BASE_URL + soup.find('a', string="Paper")['href']
                    if soup.find('a', string="Paper") else "No PDF")
        return [serial_no, title, authors, abstract, pdf_link]
    except requests.RequestException:
        return None

def main():
    paper_links = fetch_paper_links(YEAR)
    data = [extract_paper_details(link, i) for i, link in enumerate(paper_links, 1) if extract_paper_details(link, i)]
    pd.DataFrame(data, columns=["Serial No", "Title", "Authors", "Abstract", "PDF Link"]).to_csv(f"neurips_papers_{YEAR}.csv", index=False)
    print(f"✅ CSV file 'neurips_papers_{YEAR}.csv' created successfully!")

if __name__ == "__main__":
    main()
